<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/Seq2Seq_Paper_news_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import re
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
import unicodedata
import urllib3
import zipfile

In [2]:
text = pd.read_excel('/content/drive/My Drive/AI NLP 알고리즘 포트폴리오/Seq2Seq/3_문어체_뉴스(1)_200226.xlsx')
text

,ID,날짜,자동분류1,자동분류2,자동분류3,URL,언론사,원문,번역문
0,20961563,2018-10-01 00:00:00,"경제,국제경제","IT_과학,IT_과학일반","문화,문화일반",http://www.sedaily.com/NewsView/1S5QRSR5T2,서울경제,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.,Skinner's reward is mostly eye-watering.
1,20930897,2018-09-14 00:00:00,"IT_과학,IT_과학일반",NaN,NaN,http://www.sedaily.com/NewsView/1S4MSFM0IH,서울경제,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.,Even some problems can be predicted.
2,20307875,2019-04-19 00:00:00,"IT_과학,과학",NaN,NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.,Only God will exactly know why.
3,20314977,2019-04-27 00:00:00,"국제,중국","경제,국제경제",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.,Businesses should not overlook China's dispute.
4,20268904,2019-03-13 00:00:00,"문화,방송_연예","사회,미디어",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.,Slow-beating songs often float over time.
...,...,...,...,...,...,...,...,...,...
200006,2469942,20181022,"사회,노동_복지","사회,사건_사고","사회,미디어",http://www.hani.co.kr/arti/opinion/because/866...,한겨레,당시에는 경찰의 금지통보로 청와대 근처 집회가 불가능하던 시절이라 과연 경찰이 막지...,"At the time, it was a period when it was impos..."
200007,2462113,20181019,"사회,사건_사고","정치,정치일반","사회,노동_복지",http://www.hani.co.kr/arti/society/society_gen...,한겨레,양승태 대법원과 박근혜 청와대의 대표적 재판거래 의혹 사건으로 꼽히는 전국교직원노동...,The testimony of a Blue House official at the ...
200008,2463985,20181019,"경제,산업_기업","사회,사회일반",NaN,http://www.hani.co.kr/arti/society/society_gen...,한겨레,윤석열 서울중앙지검장이 19일 국정감사에 출석해 ‘적폐수사 1호’로 꼽히는 한국항공...,Seoul Central District Prosecutor's Office Chi...
200009,2463107,20181019,"사회,사건_사고","국제,유럽_EU",NaN,http://www.hani.co.kr/arti/society/society_gen...,한겨레,대법원에 재상고된 지 5년이 넘도록 재판이 지연되면서 양승태 대법원장 시절 대표적인...,The date of the ruling of the second appeal re...


In [3]:
text = text[:10000]
text = text[['원문', '번역문']]

In [4]:
ko = text['원문']
eng = text['번역문']

In [5]:
# eng = eng.apply(lambda x : '<sos> '+ x + ' <eos>')

In [6]:
eng[0] # eg = eng[i]

"Skinner's reward is mostly eye-watering."

In [7]:
ko[0] # k = ko[i]

'스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'

In [8]:
encoder_input = [k.split() for k in ko] # 단어 단위로 encoder_input에 담는다.
decoder_input = [['<sos>'] + eg.split() for eg in eng]
decoder_target = [eg.split() + ['<eos>'] for eg in eng]

In [9]:
encoder_input[0]

['스키너가', '말한', '보상은', '대부분', '눈으로', '볼', '수', '있는', '현물이다.']

In [10]:
decoder_input[0]

['<sos>', "Skinner's", 'reward', 'is', 'mostly', 'eye-watering.']

In [11]:
decoder_target[0]

["Skinner's", 'reward', 'is', 'mostly', 'eye-watering.', '<eos>']

In [12]:
# pd_decoder_input = pd.DataFrame(decoder_input)
# pd_decoder_input = pd_decoder_input.apply(lambda x: '<sos> ' + x)
# pd_decoder_input

In [13]:
decoder_input[:5]

[['<sos>', "Skinner's", 'reward', 'is', 'mostly', 'eye-watering.'],
 ['<sos>', 'Even', 'some', 'problems', 'can', 'be', 'predicted.'],
 ['<sos>', 'Only', 'God', 'will', 'exactly', 'know', 'why.'],
 ['<sos>', 'Businesses', 'should', 'not', 'overlook', "China's", 'dispute.'],
 ['<sos>', 'Slow-beating', 'songs', 'often', 'float', 'over', 'time.']]

In [14]:
decoder_target[:5]

[["Skinner's", 'reward', 'is', 'mostly', 'eye-watering.', '<eos>'],
 ['Even', 'some', 'problems', 'can', 'be', 'predicted.', '<eos>'],
 ['Only', 'God', 'will', 'exactly', 'know', 'why.', '<eos>'],
 ['Businesses', 'should', 'not', 'overlook', "China's", 'dispute.', '<eos>'],
 ['Slow-beating', 'songs', 'often', 'float', 'over', 'time.', '<eos>']]

In [15]:
# 케라스 토크나이저를 통해 단어집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거친다.

tokenizer_ko = Tokenizer(filters="", lower=False)
tokenizer_ko.fit_on_texts(encoder_input)
encoder_input_ = tokenizer_ko.texts_to_sequences(encoder_input)

tokenizer_eng = Tokenizer(filters="", lower=False)
tokenizer_eng.fit_on_texts(decoder_input)
decoder_input_ = tokenizer_eng.texts_to_sequences(decoder_input)
decoder_target_ = tokenizer_eng.texts_to_sequences(decoder_target)

In [16]:
encoder_input_ = pad_sequences(encoder_input_, padding="post")
decoder_input_ = pad_sequences(decoder_input_, padding="post")
decoder_target_ = pad_sequences(decoder_target_, padding="post")

In [17]:
decoder_target_ = pad_sequences(decoder_target_, padding='post')
decoder_target_.shape

(10000, 15)

In [18]:
encoder_input_.shape, decoder_input_.shape, decoder_target_.shape

((10000, 25), (10000, 16), (10000, 15))

In [19]:
ko_vocab_size = len(tokenizer_ko.word_index) + 1
eng_vocab_size = len(tokenizer_eng.word_index) + 1
print("한국어 단어 집합의 크기 : {:d}, 영어 단어 집합의 크기 : {:d}".format(ko_vocab_size, eng_vocab_size))

한국어 단어 집합의 크기 : 38652, 영어 단어 집합의 크기 : 22163


In [20]:
ko_to_index = tokenizer_ko.word_index
index_to_ko = tokenizer_ko.index_word # 훈련 후 결과 비교할 때 사용

eng_to_index = tokenizer_eng.word_index # 훈련 후 예측 과정에서 사용
index_to_eng = tokenizer_eng.index_word # 훈련 후 결과 비교할 때 사용

In [21]:
eng_to_index_ = eng_to_index
eng_to_index_['<eos>'] = 22163

In [22]:
indices = np.arange(encoder_input_.shape[0])
np.random.shuffle(indices)
print(indices)

[2068 9418 9413 ... 7607 2050 2307]


In [23]:
encoder_input_ = encoder_input_[indices]
decoder_input_ = decoder_input_[indices]
decoder_target_ = decoder_target_[indices]

In [24]:
# 훈련 데이터 분리

n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [25]:
encoder_input_train = encoder_input_[:-n_of_val]
decoder_input_train = decoder_input_[:-n_of_val]
decoder_target_train = decoder_target_[:-n_of_val]

encoder_input_test = encoder_input_[-n_of_val:]
decoder_input_test = decoder_input_[-n_of_val:]
decoder_target_test = decoder_target_[-n_of_val:]

In [26]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

# 차원 25, 16, 16로 조정

(6700, 25)
(6700, 16)
(6700, 15)
(3300, 25)
(3300, 16)
(3300, 15)


In [27]:
a_zero = np.zeros((6700, 1))
b_zero = np.zeros((3300, 1))

decoder_target_train = np.append(decoder_target_train, a_zero, axis=1)
decoder_target_test = np.append(decoder_target_test, b_zero, axis=1)

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(6700, 25)
(6700, 16)
(6700, 16)
(3300, 25)
(3300, 16)
(3300, 16)


In [28]:
# a_zero = np.zeros((6700, 1))
# b_zero = np.zeros((3300, 1))

# np.append(decoder_target_train, a_zero, axis=1)
# np.append(decoder_target_test, b_zero, axis=1)

# print(decoder_target_train.shape)
# print(decoder_target_test.shape)

numpy 배열 확인 및 조정(연습)

In [29]:
decoder_input_test

array([[    1,     9,   270, ...,  3098,     0,     0],
       [    1,    22,   147, ...,     0,     0,     0],
       [    1,    23,   110, ..., 15572,     0,     0],
       ...,
       [    1,   590, 18871, ...,   399,   586,     0],
       [    1,   381,   789, ...,     0,     0,     0],
       [    1,     9,   495, ...,     0,     0,     0]], dtype=int32)

In [30]:
testnp = np.arange(1, 21)
testnp

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [31]:
testnp2 = testnp.reshape(5, 4)
testnp2.shape

(5, 4)

In [32]:
testnp2.shape

(5, 4)

In [33]:
a = np.zeros((5, 1))
np.append(testnp2, a, axis=1)

# 이런식으로 decoder_target_test / train을 수동으로 zeros패딩 시켜주자.

array([[ 1.,  2.,  3.,  4.,  0.],
       [ 5.,  6.,  7.,  8.,  0.],
       [ 9., 10., 11., 12.,  0.],
       [13., 14., 15., 16.,  0.],
       [17., 18., 19., 20.,  0.]])

## Seq2Seq 기계 번역기 만들기

In [34]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [35]:
latent_dim = 50

In [36]:
# 인코더

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(ko_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [37]:
# 디코더

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(eng_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(eng_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [38]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [39]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [40]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     1932600     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     1108150     input_2[0][0]                    
_______________________________________________________________________________________

In [41]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test), batch_size = 128, epochs = 50)

Epoch 1/50
53/53 [==============================] - 9s 175ms/step - loss: 8.3686 - acc: 0.0620 - val_loss: 7.2106 - val_acc: 0.2268
Epoch 2/50
53/53 [==============================] - 7s 130ms/step - loss: 6.7498 - acc: 0.2233 - val_loss: 6.6253 - val_acc: 0.2268
Epoch 3/50
53/53 [==============================] - 7s 130ms/step - loss: 6.3497 - acc: 0.2233 - val_loss: 6.4418 - val_acc: 0.2268
Epoch 4/50
53/53 [==============================] - 7s 130ms/step - loss: 6.1283 - acc: 0.2233 - val_loss: 6.3369 - val_acc: 0.2273
Epoch 5/50
53/53 [==============================] - 7s 130ms/step - loss: 5.9821 - acc: 0.2379 - val_loss: 6.2844 - val_acc: 0.2534
Epoch 6/50
53/53 [==============================] - 7s 130ms/step - loss: 5.8755 - acc: 0.2575 - val_loss: 6.2370 - val_acc: 0.2631
Epoch 7/50
53/53 [==============================] - 7s 130ms/step - loss: 5.7905 - acc: 0.2630 - val_loss: 6.2161 - val_acc: 0.2655
Epoch 8/50
53/53 [==============================] - 7s 130ms/step - loss: 5.

In [42]:
NoneType = None
type(NoneType)

NoneType

'''
list의 변수명을 list로 한건지 확인해보자
아니면 if is 함수 통해서 알아보자
'''

In [43]:
if encoder_input_train is None:
    pass

if decoder_input_train is None:
    pass

# Nonetype문제 해결

## Seq2Seq 번역기 작동시키기

In [44]:
# 인코더

encoder_model = Model(encoder_inputs, encoder_states)

In [45]:
# 디코더

# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [46]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [47]:
def decode_sequence(input_seq):

    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = eng_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정한다.

    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_eng[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [48]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2ko(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_ko[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2eng(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=eng_to_index['<sos>']) and i!=eng_to_index['<eos>']):
            temp = temp + index_to_eng[i] + ' '
    return temp

In [49]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2ko(encoder_input_train[seq_index]))
  print("번역문 :",seq2eng(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence)
  print("\n")

원문 :  발병원인이 아직 밝혀져 있지 않아 근본적인 치료법은 없는 상태다. 
번역문 : The cause of the disease is not known yet, so there is no fundamental treatment. 
예측문 :  The market is to be used that have but it is not not


원문 :  시위에는 10대 초반의 어린 학생들까지 참여하는 등 수천명이 모였다. 
번역문 : Thousands gathered in the demonstration, including even young students in their early teens. 
예측문 :  The photo has been been found for the world with his


원문 :  발견 당시 김씨는 건강에 지장이 없는 상태였던 것으로 전해졌다. 
번역문 : Kim was reportedly not in danger of life when he was discovered. 
예측문 :  The government is not been used to make as the same


원문 :  아직 정식 개봉 전이라 서툰 소개가 걱정이 되기도 한다. 
번역문 : The introduction may be awkward, with it not yet released, is also worrisome. 
예측문 :  The government has been been been been enough for not


원문 :  삶의 목표가 분명한 것이 이렇게 좋은 것인지 미처 몰랐습니다. 
번역문 : I never realized that the obvious goal of life was so good like this. 
예측문 :  The government is not the lot of I can be to be but


